In [ ]:
import requests
import re
from bs4 import BeautifulSoup
from IPython.core.display import HTML
import pandas as pd
import math
import datetime
req = requests.get("https://store.steampowered.com/search/?specials=1&page=1")
page = req.text
soup = BeautifulSoup(page, 'html.parser')
soup

# Loads in beautifulsoup, And loads in other packages such as pandas and math.
# Afterwards the steam page is saved to req

## Getting title

In [ ]:
#Title getter
steamTitle = []


title = soup.findAll("span",{"class","title"}) # Searches for the title in the steam page
for span in title: # Loops through all of the span objects in title
    steamTitle.append(str(span.text)) 
    # Puts all titles that it finds into steamTitle
print(steamTitle)  



## Getting rating and converting to numbers

In [ ]:
#Rating getter
ratingList = []

review_summary = soup.findAll('span', {"class":"search_review_summary"}) 
# Searches for a review rating
review_summary_str = str(review_summary) 
# Converts review rating to a string
for element in review_summary: # Loops through all of the elements in review_summary
    
    reviewResult = element.attrs['data-tooltip-html'].split('<br>')

    if reviewResult[0] == 'Overwhelmingly Positive': # If reviewresult's list's index corresponds the following rating:
        ratingList.append("8 ")                     # Converts to a string number that equals the review_rating
    elif reviewResult[0] == 'Very Positive':
        ratingList.append("7 ")
    elif reviewResult[0] == 'Positive':
         ratingList.append("6 ")
    elif reviewResult[0] == 'Mostly Positive':
         ratingList.append("5 ")
    elif reviewResult[0] == 'Mixed':
         ratingList.append("4 ")
    elif reviewResult[0] == 'Mostly Negative':
         ratingList.append("3 ")
    elif reviewResult[0] == 'Negative':
         ratingList.append("2 ")
    elif reviewResult[0] == 'Very Negative' or 'Overwhelmingly Neative':
         ratingList.append("1 ")

    else:
        ratingList.append("No score ") # If you can't find a score, "No score" string is added
print(ratingList)

## Getting review numbers

In [ ]:
# Review summary getter
reviewList = []


review_summary = soup.findAll('span', {"class":"search_review_summary"})
# Searches after search_review_summary in all of the span elements
review_summary_str = str(review_summary) 
# Converts what it finds into a string

for element in review_summary: 

    reviewResult = element.attrs['data-tooltip-html'].split('<br>')

    reviewSplit = reviewResult[1].split(" ") 
# Splits away the review summary
    reviewList.append(reviewSplit[3]) 
# Puts the ammount of reviews per game into reviewList
print(reviewList)


## Getting new price and discounted price

In [ ]:
#Discounted price
priceList = []
newPrice = []


price = soup.findAll("div",{"class","search_price"})
# Searches for price in all of the divs, that contain the search_price class
for div in price:
    splitPrice = str(div.text).split('€') # Removes the € from a string
    priceList.append(re.sub(' +', ' ', splitPrice[0]).strip("\r\n"))
    # Replaces all spaces that are created in 'Free to play'


    try:
      newPrice.append(re.sub(' +', ' ', splitPrice[1]).strip("\r\n")) 

    except:
      newPrice.append(re.sub(' +', ' ', splitPrice[0]).strip("\r\n"))

print(newPrice)

## Getting date for each game

In [ ]:
#Release date getter
dateList = []

releaseDate = soup.findAll("div",{"class ","search_released"})
# Searches for release_date in all of the divs that contain the search_released tag
for div in releaseDate:
    dateList.append(str(div.text))
    # Inserts all dates into dateList and converts to a string
print(dateList)  

## Getting discount percentage

In [ ]:
#Percantage getter
percentageList = []

releaseDate = soup.findAll("div",{"class","search_discount"})
# Searches for discount in all of the divs that contain the search_discount tag

for div in releaseDate:

    if not div.text.strip(): # If the string does not contain something
        percentageList.append("No Discount") # Inserts "No discount" if no discount is found
    else: # If the string contains something
        percentageList.append(str(div.text).strip("\n")) # Inserts percentage into percentageList

print(percentageList) 

## Getting platform avaliability

In [ ]:
#Platform avaliability getter
platformListWin = []
platformListMac = []
platformListLinux = []

for platform in soup.find_all('div', {"class":"col search_name ellipsis"}):#Söker efter platformer i alla divar som har col search_name ellipsis klassen
    title = platform.find('span',attrs={'class':'title'}).text
    if platform.find('span',{"class":"win"}):
        platformListWin.append(1)
    else:
        platformListWin.append(0)

    if platform.find("span",{"class":"mac"}): # If you find MacOS
        platformListMac.append(1)
    else:
        platformListMac.append(0)

    if platform.find("span",{"class":"linux"}): # If you find Linux
        platformListLinux.append(1)
    else:
        platformListLinux.append(0)
print(platformListWin)

##  Getting current date when data was imported

In [ ]:
#When imported getter
currentDate = datetime.date.today().strftime("%d%b%Y")# Takes current date and inserts it into currentDate
currentDate

print(len(steamTitle),len(reviewList),len(ratingList),
      len( percentageList), len(dateList), len(newPrice),
      len(priceList),len(platformListWin),len(platformListLinux),
      len(platformListMac),len(currentDate ))

## Creating Dataframe and exporting to csv

In [ ]:

dataRows = None

# Creates a dataframe from all of the lists with previously scraped data
dataRows = pd.DataFrame({"Game title":steamTitle, "Reviews":reviewList,"Rating":ratingList,
                                    "Sale percentage":percentageList,
                                        'Release date':dateList, 'Pris':newPrice,
                                           'OrdinariePris':priceList,'Win':platformListWin,
                                    'Linux':platformListLinux,
                                    'OSX':platformListMac,
                                    'Tid':currentDate}) 
# Exports data to a csv
dataRows.to_csv('steamsale.csv')


dataRows.head(25)

## End of program